In [ ]:
import os
from pathlib import Path

# Vérifier que les fichiers sont bien là (optionnel mais recommandé)
dataset_dir = Path('/kaggle/input/augmented-savi-640/Dataset_B_640x640')
working_dir = Path('/kaggle/working/')
print("Contenu du dossier :")
!ls {dataset_dir}

In [ ]:
# --- Création du Fichier YAML ---

# Contenu du fichier de configuration.
# Le 'path' doit pointer vers le dossier racine du dataset.
# Les chemins 'train', 'val', 'test' sont relatifs à ce 'path'.
yaml_content = f"""
path: {dataset_dir.as_posix()}
train: images/train
val: images/val
test: images/test

names:
  0: Person
  1: Bicycle
  2: Car
  3: Cattle
"""

# Écriture du contenu dans un fichier .yaml dans le répertoire de travail
yaml_file_path = working_dir / 'dataset.yaml'
with open(yaml_file_path, 'w') as f:
    f.write(yaml_content)

print(f"Fichier de configuration créé avec succès à l'emplacement : {yaml_file_path}")
print("\n--- Contenu du YAML ---")
!cat {yaml_file_path}

In [ ]:
# --- Installation ---
# On installe la bibliothèque ultralytics qui contient l'implémentation de YOLOv8.
# Le flag '-q' (quiet) permet de réduire la quantité de logs durant l'installation.
!pip install ultralytics -q

print("Installation terminée.")

In [ ]:
import torch
import torch.nn as nn

class ChannelAttention(nn.Module):
    """Channel-attention module https://github.com/open-mmlab/mmdetection/tree/v3.0.0rc1/configs/rtmdet."""

    def __init__(self, channels: int) -> None:
        """Initializes the class and sets the basic configurations and instance variables required."""
        super().__init__()
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Conv2d(channels, channels, 1, 1, 0, bias=True)
        self.act = nn.Sigmoid()

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Applies forward pass using activation on convolutions of the input, optionally using batch normalization."""
        return x * self.act(self.fc(self.pool(x)))


class SpatialAttention(nn.Module):
    """Spatial-attention module."""

    def __init__(self, kernel_size=7):
        """Initialize Spatial-attention module with kernel size argument."""
        super().__init__()
        assert kernel_size in {3, 7}, "kernel size must be 3 or 7"
        padding = 3 if kernel_size == 7 else 1
        self.cv1 = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.act = nn.Sigmoid()

    def forward(self, x):
        """Apply channel and spatial attention on input for feature recalibration."""
        return x * self.act(self.cv1(torch.cat([torch.mean(x, 1, keepdim=True), torch.max(x, 1, keepdim=True)[0]], 1)))


class CBAM(nn.Module):
    """Convolutional Block Attention Module."""

    def __init__(self, c1, kernel_size=7):
        """Initialize CBAM with given input channel (c1) and kernel size."""
        super().__init__()
        self.channel_attention = ChannelAttention(c1)
        self.spatial_attention = SpatialAttention(kernel_size)

    def forward(self, x):
        """Applies the forward pass through C1 module."""
        return self.spatial_attention(self.channel_attention(x))

print("Module CBAM définis avec succès.")

In [ ]:
# --- Étape 1 : Importer le parseur de modèles ---
from ultralytics.nn import tasks

# --- Enregistrer notre module personnalisé ---
tasks.CBAM = CBAM
print("Module CBAM enregistré avec succès.")

# --- Création du Fichier de Configuration YAML Final ---

yaml_config_content = """
# Ultralytics YOLO 🚀, AGPL-3.0 license
# Fichier de configuration pour YOLOv8s avec des blocs C2f_CBAM

# Paramètres
nc: 4 
scales:
  # [depth, width, max_channels]
  s: [0.33, 0.50, 1024]  #

backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]]  # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]]  # 1-P2/4
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]]  # 3-P3/8
  - [-1, 6, C2f, [256, True]]
  - [-1, 1, Conv, [512, 3, 2]]  # 5-P4/16
  - [-1, 6, C2f, [512, True]]
  - [-1, 1, Conv, [1024, 3, 2]]  # 7-P5/32
  - [-1, 3, C2f, [1024, True]]
  - [-1, 1, SPPF, [1024, 5]]  # 9

head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]  # 10
  - [-1, 1, CBAM, [512]]  # Add CBAM after Upsample
  - [[-1, 6], 1, Concat, [1]]  # 12 cat backbone P4
  - [-1, 3, C2f, [512, False]]  # 13

  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]  # 14
  - [-1, 1, CBAM, [256]]  # Add CBAM after Upsample
  - [[-1, 4], 1, Concat, [1]]  # 16 cat backbone P3
  - [-1, 3, C2f, [256, False]]  # 17

  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]  # 18
  - [-1, 1, CBAM, [128]]  # Add CBAM after Upsample
  - [[-1, 2], 1, Concat, [1]]  # 20 cat backbone P2
  - [-1, 1, C2f, [128, False]]  # 21

  - [-1, 1, Conv, [128, 3, 2]]  # 22
  - [[-1, 17], 1, Concat, [1]]  # 23 cat head P3
  - [-1, 3, C2f, [256, False]]  # 24

  - [-1, 1, Conv, [256, 3, 2]]  # 25
  - [[-1, 13], 1, Concat, [1]]  # 26 cat head P4
  - [-1, 3, C2f, [512, False]]  # 27

  - [-1, 1, Conv, [512, 3, 2]]  # 28
  - [[-1, 9], 1, Concat, [1]]  # 29 cat head P5
  - [-1, 3, C2f, [1024, False]]  # 30

  - [[21, 24, 27, 30], 1, Detect, [nc]]  # 31 Detect(P2, P3, P4, P5)
"""

# Écrire ce contenu dans un fichier .yaml dans le répertoire de travail
custom_yaml_path = working_dir / 'yolov8s-cbam.yaml'
with open(custom_yaml_path, 'w') as f:
    f.write(yaml_config_content)

print(f"Fichier de configuration YAML personnalisé créé : {custom_yaml_path}")

In [ ]:
# --- Lancement de l'Entraînement ---
from ultralytics import YOLO

# 1. Charger le modèle en utilisant notre configuration YAML personnalisée.
#    Le framework va lire le YAML, voir qu'il a besoin d'un module 'CBAM',
#    et le trouvera automatiquement car nous l'avons défini dans la Cellule 2.
model = YOLO(custom_yaml_path)

# 2. Charger les poids pré-entraînés du modèle 's' standard.
#    Le framework fera correspondre les poids des couches qui existent dans les deux modèles.
model.load('yolov8s.pt')

# 3. Lancer l'entraînement
print("\\nLancement de l'entraînement du modèle YOLOv8s + CBAM...")
results = model.train(
    data=str(yaml_file_path),
    epochs=100,
    imgsz=640,
    batch=8,
    name='yolov8s_cbam_backbone_yaml' # Nouveau nom pour ne pas écraser l'ancienne tentative
)

print("Entraînement terminé.")

In [ ]:
from IPython.display import Image, display

# --- Visualisation des Résultats ---

# Chemin vers le dossier des résultats (le nom est celui défini dans model.train)
results_dir = Path('/kaggle/working/runs/detect/yolov8s_cbam_backbone')

# Afficher les graphiques des métriques et des pertes (loss)
print("--- Courbes de performance (métriques et pertes) ---")
display(Image(filename=results_dir / 'results.png', width=800))

# Afficher la matrice de confusion
print("\n--- Matrice de confusion ---")
display(Image(filename=results_dir / 'confusion_matrix.png', width=600))

# Afficher un batch de prédictions sur l'ensemble de validation
print("\n--- Exemples de prédictions sur le set de validation ---")
display(Image(filename=results_dir / 'val_batch0_pred.jpg', width=800))